In [10]:
import joblib
import numpy as np
import pandas as pd

In [63]:
# Load the joblib model
model_file = joblib.load('xgboost_opt.joblib')
model = model_file['model']
feature_encoder = model_file['feature_encoder']
scaler = model_file['scaler']


In [64]:
pricing = pd.read_csv('get_around_pricing_project.csv',index_col=0)
pricing_without_target = pricing.iloc[:,:-1]
input1 = pricing_without_target.iloc[0].to_dict()
input2 = pricing_without_target.iloc[1].to_dict()
# Input data should be resembling to this:
input_data = {'car_criteria':[input1,input2]}
print(input_data)

{'car_criteria': [{'model_key': 'Citroën', 'mileage': 140411, 'engine_power': 100, 'fuel': 'diesel', 'paint_color': 'black', 'car_type': 'convertible', 'private_parking_available': True, 'has_gps': True, 'has_air_conditioning': False, 'automatic_car': False, 'has_getaround_connect': True, 'has_speed_regulator': True, 'winter_tires': True}, {'model_key': 'Citroën', 'mileage': 13929, 'engine_power': 317, 'fuel': 'petrol', 'paint_color': 'grey', 'car_type': 'convertible', 'private_parking_available': True, 'has_gps': True, 'has_air_conditioning': False, 'automatic_car': False, 'has_getaround_connect': False, 'has_speed_regulator': True, 'winter_tires': True}]}


In [65]:
# Define preprocessing:

def preprocess_data(input_data):
    # Create the list that will contain the preprocessed options:
    preprocessed_data = []
    for option in input_data['car_criteria']:
        # Put each option of criteria into a dataframe
        option_df = pd.DataFrame([option])
        # Preprocess this dataframe
        preprocessed_option_df = feature_encoder.transform(option_df)
        preprocessed_option_array = preprocessed_option_df.toarray()
        # Add preprocessed option to the final list:
        preprocessed_data.append(preprocessed_option_array)
  
    # Convert the list of preprocessed data to a numpy array
    preprocessed_data = np.concatenate(preprocessed_data, axis=0)

    return preprocessed_data

In [68]:
# Defining predictions:

def predict_data(model, preprocessed_data):

    normalized_predictions = model.predict(preprocessed_data)
    # Reshape the normalized predictions if needed
    if len(normalized_predictions.shape) == 1:
        normalized_predictions = normalized_predictions.reshape(-1, 1)
    
    # Inverse transform the predictions to the original scale
    predictions = scaler.inverse_transform(normalized_predictions)

    return predictions

In [69]:
# IN THE APP:

# Preprocess the input data
preprocessed_data = preprocess_data(input_data)

# Make predictions using the loaded model and preprocessed data
predictions = predict_data(model, preprocessed_data)

# Print the predictions
print(predictions)

[[ 87.92895]
 [155.97945]]


In [73]:
# Check the versions of scikit-learn and joblib
import sklearn
print(f"scikit-learn version: {sklearn.__version__}")
print(f"joblib version: {joblib.__version__}")


scikit-learn version: 1.0.2
joblib version: 1.2.0


In [ ]:
$ curl -i -H "Content-Type: application/json" -X POST -d '{'car_options': [{'model_key': 'Citroën', 'mileage': 140411, 'engine_power': 100, 'fuel': 'diesel', 'paint_color': 'black', 'car_type': 'convertible', 'private_parking_available': True, 'has_gps': True, 'has_air_conditioning': False, 'automatic_car': False, 'has_getaround_connect': True, 'has_speed_regulator': True, 'winter_tires': True}, {'model_key': 'Citroën', 'mileage': 13929, 'engine_power': 317, 'fuel': 'petrol', 'paint_color': 'grey', 'car_type': 'convertible', 'private_parking_available': True, 'has_gps': True, 'has_air_conditioning': False, 'automatic_car': False, 'has_getaround_connect': False, 'has_speed_regulator': True, 'winter_tires': True}]}' https://getaround-api-elo-16ab161d9781.herokuapp.com/predict

In [ ]:
curl -i -H "Content-Type: application/json" -X POST -d "{'car_options': [{'model_key': 'Citroën', 'mileage': 140411, 'engine_power': 100, 'fuel': 'diesel', 'paint_color': 'black', 'car_type': 'convertible', 'private_parking_available': True, 'has_gps': True, 'has_air_conditioning': False, 'automatic_car': False, 'has_getaround_connect': True, 'has_speed_regulator': True, 'winter_tires': True}, {'model_key': 'Citroën', 'mileage': 13929, 'engine_power': 317, 'fuel': 'petrol', 'paint_color': 'grey', 'car_type': 'convertible', 'private_parking_available': True, 'has_gps': True, 'has_air_conditioning': False, 'automatic_car': False, 'has_getaround_connect': False, 'has_speed_regulator': True, 'winter_tires': True}]}" https://getaround-api-elo-16ab161d9781.herokuapp.com/predict

In [74]:
import requests

response = requests.post("https://getaround-api-elo-16ab161d9781.herokuapp.com/predict", json={'car_options': [{'model_key': 'Citroën', 'mileage': 140411, 'engine_power': 100, 'fuel': 'diesel', 'paint_color': 'black', 'car_type': 'convertible', 'private_parking_available': True, 'has_gps': True, 'has_air_conditioning': False, 'automatic_car': False, 'has_getaround_connect': True, 'has_speed_regulator': True, 'winter_tires': True}, {'model_key': 'Citroën', 'mileage': 13929, 'engine_power': 317, 'fuel': 'petrol', 'paint_color': 'grey', 'car_type': 'convertible', 'private_parking_available': True, 'has_gps': True, 'has_air_conditioning': False, 'automatic_car': False, 'has_getaround_connect': False, 'has_speed_regulator': True, 'winter_tires': True}]})
print(response.json())

{'predictions': ['Option 1: 88 €', 'Option 2: 156 €']}
